# Análisis de campañas de alivio financiero y propuesta para recuperación de cartera vencida en una entidad financiera

## Breve descripción del proyecto

En el presente proyecto, se realiza el análisis de los datos correspondientes a una campaña que realiza la entidad financiera, a la cual se la referirá mediante el nombre ficticio ***Mi Banco Feliz***, cuyo objetivo es lanzar 2 campañas en pro de sus clientes en estado de mora a fin de recuperar más de un 60% de la cartera vencida que se mantiene con corte a la fecha de este proyecto, es decir, Junio de 2025.

Se van a responder a las siguientes preguntas para poder entender de mejor manera cuál es el problema actual y de que manera se podría brindar una solución que satizfaga tanto las necesidades del Banco sin descuidar el bienestar del cliente:

1. ¿Qué perfiles de clientes están siendo atendidos por cada campaña y canal?
2. ¿Cómo podría optimizarse el enfoque actual para mejorar la efectividad de las campañas?

**Nota:** *A lo largo del presente reporte se analizan datos reales, mismos que han sido debidamente ofuscados y/o omitidos intencionalmente con el fin de proteger posible información sensible, así como por mantener respeto y apego hacia las leyes de protección de datos tanto locales como internacionales.*

## Carga del conjunto de datos en Pandas

Se importa la librería *pandas*, se carga el dataset y se muestran sus parámetros descriptivos incluyendo sus primeras filas, información general de tipos de datos, evaluación inicial de valores nulos, así como estadísticas descriptivas preliminares que otorga esta librería.

In [1]:
import pandas as pd

In [2]:
df_morosidad = pd.read_csv("df_morosidad.csv")

In [3]:
df_morosidad.head()

,codigo_cliente,rango_morosidad,monto_credito,monto_por_vencer,monto_vencido,intereses,producto_host,dias_mora,gestor,recibe_sueldo_fijo,region,sector_general
0,DNI-9502992,31 - 60 DIAS,8045.23,5686.38,0.00,143.94,PRECISO,36,RED COMERCIAL,SÍ,SIERRA,SECTOR PÚBLICO
1,DNI-5110681,16 - 30 DIAS,5924.60,0.00,92.60,98.97,PRECISO,22,DIGITAL,NO,SIERRA,TRANSPORTE Y LOGÍSTICA
2,DNI-6527606,6 - 15 DIAS,5125.55,0.00,74.08,148.94,PRECISO,10,DIGITAL,NO,COSTA,COMERCIO
3,DNI-5095519,16 - 30 DIAS,4848.91,0.00,142.72,65.76,PRECISO,23,DIGITAL,NO,SIERRA,COMERCIO
4,DNI-9856629,16 - 30 DIAS,6621.77,5462.92,0.00,127.39,PRECISO,28,RED COMERCIAL,NO,SIERRA,SERVICIOS


In [4]:
df_morosidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   codigo_cliente      4115 non-null   object 
 1   rango_morosidad     4115 non-null   object 
 2   monto_credito       4115 non-null   float64
 3   monto_por_vencer    4115 non-null   float64
 4   monto_vencido       4115 non-null   float64
 5   intereses           4115 non-null   float64
 6   producto_host       4115 non-null   object 
 7   dias_mora           4115 non-null   int64  
 8   gestor              4115 non-null   object 
 9   recibe_sueldo_fijo  4115 non-null   object 
 10  region              4115 non-null   object 
 11  sector_general      4115 non-null   object 
dtypes: float64(4), int64(1), object(7)
memory usage: 385.9+ KB


In [5]:
df_morosidad.describe()

,monto_credito,monto_por_vencer,monto_vencido,intereses,dias_mora
count,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000
mean,9889.804168,3742.993896,62.615878,165.204522,26.249332
std,11221.651826,7083.584846,135.121374,181.270760,13.370148
min,1124.280000,0.000000,0.000000,9.210000,10.000000
25%,4401.505000,0.000000,0.000000,60.200000,15.000000
50%,7072.150000,1493.760000,0.000000,113.110000,23.000000
75%,11161.935000,5186.860000,90.080000,204.440000,35.000000
max,209131.960000,138940.420000,3322.440000,2439.470000,60.000000
